In [1]:
import gym
import gym_anytrading
from finta import TA
from gym_anytrading.envs import StocksEnv, ForexEnv

from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import A2C

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/Users/baskjellvandesand/opt/anaconda3/envs/test_env/lib/python3.6/site-packages/stable_baselines/__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation."


In [2]:
df = pd.read_csv("data/DAX3015.csv")

df['timestamp'] = df['Date'].astype(str) + "." + df['Time'].astype(str)
del df['Time']
del df['Date']
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

df["Volume"] = df["Volume"].astype(float)

df = df.between_time('9:00', '18:00')

df['SMA12'] = TA.SMA(df, 12)
df['SMA50'] = TA.SMA(df, 50)
df['SMA200'] = TA.SMA(df, 200)
df['RSI'] = TA.RSI(df)
df['OBV'] = TA.OBV(df)



df.fillna(0, inplace=True)
df.head(10)

,Open,High,Low,Close,Volume,SMA12,SMA50,SMA200,RSI,OBV
timestamp,,,,,,,,,,
2020-10-06 14:00:00,12838.2,12851.0,12834.7,12840.5,951.0,0.0,0.0,0.0,0.000000,0.0
2020-10-06 14:15:00,12840.7,12878.3,12839.3,12874.1,995.0,0.0,0.0,0.0,100.000000,995.0
2020-10-06 14:30:00,12874.6,12906.9,12874.6,12894.7,1391.0,0.0,0.0,0.0,100.000000,2386.0
2020-10-06 14:45:00,12895.2,12904.7,12886.8,12889.9,998.0,0.0,0.0,0.0,90.926276,1388.0
2020-10-06 15:00:00,12890.4,12903.9,12873.4,12902.3,1080.0,0.0,0.0,0.0,92.755138,2468.0
2020-10-06 15:15:00,12902.8,12934.0,12901.1,12925.7,1219.0,0.0,0.0,0.0,94.860390,3687.0
2020-10-06 15:30:00,12925.2,12943.8,12924.1,12932.1,1439.0,0.0,0.0,0.0,95.265607,5126.0
2020-10-06 15:45:00,12933.1,12946.0,12930.0,12944.8,1068.0,0.0,0.0,0.0,95.948270,6194.0
2020-10-06 16:00:00,12945.3,12958.7,12935.9,12944.0,1239.0,0.0,0.0,0.0,95.018827,4955.0


In [3]:
env = gym.make('stocks-v0', frame_bound=(50, 100), window_size=10)

In [4]:
def add_signals(env):

    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Close', 'Open', 'High', 'Low', 'Volume', 'SMA12', 'SMA50', 'SMA200', 'RSI', 'OBV']].to_numpy()[start:end]
    return prices, signal_features


In [5]:
class MyCustomEnv(ForexEnv):
    _process_data = add_signals

env2 = MyCustomEnv(df=df, window_size=5, frame_bound=(210,400))

In [6]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
model = A2C('MlpLstmPolicy', env, verbose=1) 
model.learn(total_timesteps=1000000)





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


---------------------------------
| explained_variance | 3.58e-07 |
| fps                | 2        |
| nupdates           | 1        |
| policy_entropy     | 0.693    |
| total_timesteps    | 5        |
| value_loss         | 1.12e+08 |
---------------------------------
---------------------------------
| explained_variance | 0        |
| fps                | 91       |
| nupdates           | 100      |
| policy_entropy     | 0.693    |
| total_timesteps    | 500      |
| value_loss         | 2.53e+10 |
---------------------------------
---------------------------------
| explained_variance | 0        |
| fps                | 113     

----------------------------------
| explained_variance | -1.55e-06 |
| fps                | 135       |
| nupdates           | 1400      |
| policy_entropy     | 0.47      |
| total_timesteps    | 7000      |
| value_loss         | 1.25e+08  |
----------------------------------
---------------------------------
| explained_variance | 1.21e-05 |
| fps                | 135      |
| nupdates           | 1500     |
| policy_entropy     | 0.476    |
| total_timesteps    | 7500     |
| value_loss         | 2.42e+07 |
---------------------------------
---------------------------------
| explained_variance | 1.79e-07 |
| fps                | 135      |
| nupdates           | 1600     |
| policy_entropy     | 0.333    |
| total_timesteps    | 8000     |
| value_loss         | 1.7e+10  |
---------------------------------
---------------------------------
| explained_variance | -10.2    |
| fps                | 134      |
| nupdates           | 1700     |
| policy_entropy     | 0.0822   |
| tota

---------------------------------
| explained_variance | 1.26e-05 |
| fps                | 128      |
| nupdates           | 4500     |
| policy_entropy     | 0.439    |
| total_timesteps    | 22500    |
| value_loss         | 2.18e+09 |
---------------------------------
----------------------------------
| explained_variance | -5.49e+03 |
| fps                | 128       |
| nupdates           | 4600      |
| policy_entropy     | 0.0022    |
| total_timesteps    | 23000     |
| value_loss         | 0.000276  |
----------------------------------
---------------------------------
| explained_variance | -0.666   |
| fps                | 127      |
| nupdates           | 4700     |
| policy_entropy     | 0.00596  |
| total_timesteps    | 23500    |
| value_loss         | 0.000108 |
---------------------------------
---------------------------------
| explained_variance | -24.2    |
| fps                | 127      |
| nupdates           | 4800     |
| policy_entropy     | 0.00213  |
| tota

---------------------------------
| explained_variance | -2.75    |
| fps                | 125      |
| nupdates           | 7600     |
| policy_entropy     | 0.0225   |
| total_timesteps    | 38000    |
| value_loss         | 0.000121 |
---------------------------------
---------------------------------
| explained_variance | -238     |
| fps                | 125      |
| nupdates           | 7700     |
| policy_entropy     | 7.93e-05 |
| total_timesteps    | 38500    |
| value_loss         | 9.79e-06 |
---------------------------------
---------------------------------
| explained_variance | -0.141   |
| fps                | 125      |
| nupdates           | 7800     |
| policy_entropy     | 8.4e-05  |
| total_timesteps    | 39000    |
| value_loss         | 2.07e-06 |
---------------------------------
----------------------------------
| explained_variance | -1.87e+03 |
| fps                | 125       |
| nupdates           | 7900      |
| policy_entropy     | 3.46e-05  |
| total_t

---------------------------------
| explained_variance | -0.134   |
| fps                | 124      |
| nupdates           | 10600    |
| policy_entropy     | 0.000184 |
| total_timesteps    | 53000    |
| value_loss         | 1.18e-06 |
---------------------------------
---------------------------------
| explained_variance | -52.2    |
| fps                | 124      |
| nupdates           | 10700    |
| policy_entropy     | 0.00655  |
| total_timesteps    | 53500    |
| value_loss         | 0.00132  |
---------------------------------
----------------------------------
| explained_variance | -3.41e+06 |
| fps                | 124       |
| nupdates           | 10800     |
| policy_entropy     | 9.75e-05  |
| total_timesteps    | 54000     |
| value_loss         | 3.37e-05  |
----------------------------------
---------------------------------
| explained_variance | -0.117   |
| fps                | 124      |
| nupdates           | 10900    |
| policy_entropy     | 0.000153 |
| tota

---------------------------------
| explained_variance | -0.101   |
| fps                | 124      |
| nupdates           | 13700    |
| policy_entropy     | 0.0003   |
| total_timesteps    | 68500    |
| value_loss         | 9.3e-06  |
---------------------------------
---------------------------------
| explained_variance | -382     |
| fps                | 124      |
| nupdates           | 13800    |
| policy_entropy     | 0.00191  |
| total_timesteps    | 69000    |
| value_loss         | 0.000703 |
---------------------------------
---------------------------------
| explained_variance | -2.7     |
| fps                | 124      |
| nupdates           | 13900    |
| policy_entropy     | 0.00025  |
| total_timesteps    | 69500    |
| value_loss         | 8.12e-05 |
---------------------------------
---------------------------------
| explained_variance | -0.119   |
| fps                | 124      |
| nupdates           | 14000    |
| policy_entropy     | 0.000369 |
| total_timest

---------------------------------
| explained_variance | -0.131   |
| fps                | 123      |
| nupdates           | 16800    |
| policy_entropy     | 0.000167 |
| total_timesteps    | 84000    |
| value_loss         | 3.44e-05 |
---------------------------------
---------------------------------
| explained_variance | -588     |
| fps                | 123      |
| nupdates           | 16900    |
| policy_entropy     | 0.000283 |
| total_timesteps    | 84500    |
| value_loss         | 0.00203  |
---------------------------------
---------------------------------
| explained_variance | -9.49    |
| fps                | 123      |
| nupdates           | 17000    |
| policy_entropy     | 0.000249 |
| total_timesteps    | 85000    |
| value_loss         | 0.000209 |
---------------------------------
---------------------------------
| explained_variance | -0.226   |
| fps                | 123      |
| nupdates           | 17100    |
| policy_entropy     | 0.000244 |
| total_timest

In [ ]:
env = MyCustomEnv(df=df, window_size=5, frame_bound=(210,400))
obs = env.reset()
while True: 
    obs = obs[np.newaxis, ...]
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        print("info", info)
        break

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()